In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import arviz as az

In [9]:
from sim.dy import Model
from sim import load_inputs
import sim.dy.keys as I

In [2]:
out_path = f'out/dy_free'

In [3]:
pars_post = json.load(open(f'{out_path}/Post.json', 'r'))

In [10]:
inputs = load_inputs('data/pars.json')

m = Model(inputs, year0=1970)

In [12]:
p0 = pars_post[0]
y0, ms, _ = m.simulate_to_baseline(p0)

In [15]:
y = y0.reshape((-1, 2))

In [18]:
y[I.LTBI].sum(0) / y.sum()

array([0.16302566, 0.10923832])

In [19]:
_, ms, _ = m.simulate(p0)

In [23]:
p0['p_comorb']

0.2990742713658743

In [24]:
y.sum(0) / y.sum()

array([0.70092573, 0.29907427])

# MU

In [25]:
r_acf = 0.01

In [27]:
sens_cxr, spec_cxr = p0['acf_cxr_sens'], p0['acf_cxr_spec']
sens_xpert, spec_xpert = p0['acf_xpert_sens'], p0['acf_xpert_spec']

In [60]:
p_tp = sens_cxr * sens_xpert
p_fn = 1 - p_tp
p_fp = 1 - spec_cxr + (1 - spec_cxr) * spec_xpert
p_tn = spec_cxr + (1 - spec_cxr) * spec_xpert


reach0 = r_acf * y
reach0[I.Tx_DS] = 0
reach0[I.Tx_DR] = 0

reach1 = np.zeros_like(reach0) 
reach1[I.U] = (1 - spec_cxr)
reach1[I.LTBI] = (1 - spec_cxr)
reach1[I.Infectious] = sens_cxr
reach1 *= reach0

reach2 = np.zeros_like(reach1)
reach2[I.U] = (1 - spec_xpert)
reach2[I.LTBI] = (1 - spec_xpert)
reach2[I.Infectious] = sens_xpert
reach2 *= reach1

ppv_mu = reach2[I.Infectious].sum() / reach2.sum()

reach0.sum(), reach1.sum(), reach2.sum(), ppv_mu

(23794.05162045836, 5689.689150826871, 285.3128546132116, 0.40598963228827195)

In [51]:
from scipy.optimize import minimize_scalar

In [78]:
def fn(spec, ppv):
    reach0 = r_acf * y
    reach0[I.Tx_DS] = 0
    reach0[I.Tx_DR] = 0

    reach1 = np.zeros_like(reach0) 

    reach1[I.LTBI + I.Asym] = (1 - spec)
    reach1[I.U] = (1 - spec)
    reach1[I.Sym + I.ExSym] = 1
    reach1 *= reach0

    reach2 = np.zeros_like(reach1)
    reach2[I.U] = (1 - spec_xpert)
    reach2[I.LTBI] = (1 - spec_xpert)
    reach2[I.Infectious] = sens_xpert
    reach2 *= reach1

    ppv_d2d = reach2[I.Infectious].sum() / reach2.sum()
    
    return (ppv_d2d - ppv) ** 2

opt = minimize_scalar(fn, 1, args=(ppv_mu, ), method='bounded', bounds=(0.5, 1))
opt.x

0.4059899194238976